<a href="https://colab.research.google.com/github/JPUG26/proyecto_Kaggle_Modelos_I/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 99 - modelo solución


In [2]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

## Descargar archivo kaggle.json

Crear un archivo `kaggle.json` con su token de autenticación, subirlo a este espacio de trabajo del cuaderno y ejecutar la siguiente celda.

In [3]:
# Importar librería os (Operating System) de Python para modificar variables de entorno o gestionar archivos
import os

# Le indica a Kaggle dónde debe buscar el archivo de autenticación
os.environ['KAGGLE_CONFIG_DIR'] = '.'

# Comando de terminal linux para que solo el propietario pueda leer y escribir
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


## Descomprimir e inspeccionar datos

In [29]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [30]:
!wc *.csv

     7703     24674    501013 Consolidado.csv
   382949    382949   4785755 my_submission.csv
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1676727  15936194 212921116 total


# Se agrega al dataframe train la desviación estándar, puntaje  y la cantidad de estudiantes por programa académico y por departamento que presentaron las pruebas saber pro entre 2018 y 2021

In [160]:
import pandas as pd
import unicodedata

"""
 id del archivo que contiene la desviación estándar, el puntaje promedio y la cantidad de estudiantes por programa académico y por departamento
 que presentaron las pruebas saber pro entre 2018 y 2021, los datos extraidos de los enlaces de abajo fueron filtrados y consolidados en un solo
 archivo csv que se integró como tres nuevas columnas al dataset.

 Enlaces a los archivos con los que se construyó el consolidado
 https://www.icfes.gov.co/wp-content/uploads/2024/12/Resultados-agregados-saber-pro-2018-1.xlsx
 https://www.icfes.gov.co/wp-content/uploads/2024/12/Base-de-datos-de-resultados-agregados-de-saber-pro-2019-1.xlsx
 https://www.icfes.gov.co/wp-content/uploads/2025/06/Base-de-datos-de-resultados-agregados-de-saber-pro-2020.xlsx
 https://www.icfes.gov.co/wp-content/uploads/2025/06/Base-de-datos-de-resultados-agregados-de-saber-pro-2021.xlsx
"""
file_id = '1ZAkIYpc0w0pAiER0OotuW0uyljl-1a_I'

# Url del archivo con los datos adicionales
url_datos_agregados = f'https://drive.google.com/uc?export=download&id={file_id}'

#Elimina tildes, espacios extra y convierte a mayúsculas para estandarizar.
def normalizar_texto(texto):
  # Si el valor no es texto, retorna un valor por defecto para evitar errores
    if not isinstance(texto, str):
      #Devuelve el valor por defecto si no es texto
        return "DESCONOCIDO"
    #Eliminación de tildes y caracteres especiales
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    #Pasa todo a mayúsculas y elimina espacios en blanco al inicio y al final
    return texto.upper().strip()

# Carga el archivo 'train.csv' en un DataFrame separando los datos por comas
df_train = pd.read_csv('train.csv', sep=',')

# Este archivo no tiene cabeceras, así que se asignan nombres manuales
col_names_consolidado = ['ANIO_CONSOL', 'DEPTO_CONSOL', 'PROG_CONSOL', 'VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3']

# Carga datos desde la URL y asigna nombres manuales, ya que el archivo no tiene cabeceras
df_consol = pd.read_csv(url_datos_agregados, header=None, names=col_names_consolidado)

# Extrae los primeros 4 dígitos (el año) del periodo y los convierte a entero
df_train['ANIO_MATCH'] = df_train['PERIODO_ACADEMICO'].astype(str).str[:4].astype(int)

# Aplica la función de limpieza para estandarizar los nombres del departamento en el dataframe train
df_train['DEPTO_MATCH'] = df_train['E_PRGM_DEPARTAMENTO'].apply(normalizar_texto)

# Aplica la función de limpieza para estandarizar los nombres del programa académico en el dataframe train
df_train['PROG_MATCH'] = df_train['E_PRGM_ACADEMICO'].apply(normalizar_texto)

# Aplica la función de limpieza para estandarizar los nombres del departamento en el dataframe consolidado que contiene los datos adicionales
df_consol['DEPTO_MATCH'] = df_consol['DEPTO_CONSOL'].apply(normalizar_texto)

# Aplica la función de limpieza para estandarizar los nombres del programa academico en el dataframe consolidado que contiene los datos adicionales
df_consol['PROG_MATCH'] = df_consol['PROG_CONSOL'].apply(normalizar_texto)

# Convierte el año a entero para asegurar la coincidencia en el cruce
df_consol['ANIO_MATCH'] = df_consol['ANIO_CONSOL'].astype(int)

# Cruza las tablas (Left Join) usando las claves generadas para agregar las variables del archivo consolidado
df_merged = pd.merge(
    df_train,
    df_consol[['ANIO_MATCH', 'DEPTO_MATCH', 'PROG_MATCH', 'VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3']],
    on=['ANIO_MATCH', 'DEPTO_MATCH', 'PROG_MATCH'],
    how='left'
)

# Elimina las columnas auxiliares que se crearon solo para realizar el cruce
df_final = df_merged.drop(columns=['ANIO_MATCH', 'DEPTO_MATCH', 'PROG_MATCH'])

print(f"Dimensiones de df_final antes de eliminar duplicados de ID: {df_final.shape}")
# Eliminar filas duplicadas basadas en 'ID' para asegurar IDs únicos para la submission
df_final.drop_duplicates(subset='ID', keep='first', inplace=True)

# Mostrar resultados
print(f"Dimensiones de df_final después de eliminar duplicados de ID: {df_final.shape}")
print(f"Dimensiones originales: {df_train.shape}")
print(f"Dimensiones finales (tras limpieza y deduplicación): {df_final.shape}")
print("Columnas nuevas agregadas:", ['VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3'])
print(df_final.head())

Dimensiones de df_final antes de eliminar duplicados de ID: (895146, 24)
Dimensiones de df_final después de eliminar duplicados de ID: (692500, 24)
Dimensiones originales: (692500, 24)
Dimensiones finales (tras limpieza y deduplicación): (692500, 24)
Columnas nuevas agregadas: ['VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3']
       ID  PERIODO_ACADEMICO            E_PRGM_ACADEMICO E_PRGM_DEPARTAMENTO  \
0  904256              20212                  ENFERMERIA              BOGOTÁ   
1  645256              20212                     DERECHO           ATLANTICO   
3  308367              20203       MERCADEO Y PUBLICIDAD              BOGOTÁ   
4  470353              20195  ADMINISTRACION DE EMPRESAS           SANTANDER   
6  989032              20212                  PSICOLOGIA           ANTIOQUIA   

                E_VALORMATRICULAUNIVERSIDAD E_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones    Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                

# Configuración Inicial e Importación de Librerías

In [161]:
import pandas as pd
import numpy as np
import unicodedata
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder, TargetEncoder, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

# Agrupación por áreas de conocimiento de la columna PROGRAMA_ACADEMICO
Para reducir la complejidad de la columna PROGRAMA_ACADEMICO (que tiene demasiados valores únicos), se crea un diccionario que agrupa las carreras en 9 áreas (educación, Ciencias de la Salud, Ingeniería y Tecnología, Ciencias Sociales y Humanas, Artes, Diseño y Comunicación, Ciencias Básicas y Naturales, Ciencias Agropecuarias y Ambientales y Otras Áreas)

In [162]:
# --- DICCIONARIO DE ÁREAS ---
MAPEO_AREAS = {
    # --- 1. Educación ---
    "LICENCIATURA": "Educación (Licenciaturas)", "PEDAGOGIA": "Educación (Licenciaturas)",
    "ETNOEDUCACION": "Educación (Licenciaturas)",
    # --- 2. Ciencias de la Salud ---
    "ENFERMERIA": "Ciencias de la Salud", "MEDICINA": "Ciencias de la Salud",
    "ODONTOLOGIA": "Ciencias de la Salud", "PSICOLOGIA": "Ciencias de la Salud",
    "SALUD": "Ciencias de la Salud", "BACTERIOLOGIA": "Ciencias de la Salud",
    "FISIOTERAPIA": "Ciencias de la Salud", "NUTRICION": "Ciencias de la Salud",
    "INSTRUMENTACION QUIRURGICA": "Ciencias de la Salud", "OPTOMETRIA": "Ciencias de la Salud",
    "FONOAUDIOLOGIA": "Ciencias de la Salud", "TERAPIA": "Ciencias de la Salud",
    "VETERINARIA": "Ciencias de la Salud", "ZOOTECNIA": "Ciencias de la Salud",
    "BIOINGENIERIA": "Ciencias de la Salud", "BIOMEDICA": "Ciencias de la Salud",
    "GERONTOLOGIA": "Ciencias de la Salud",
    # --- 3. Ingeniería y Tecnología ---
    "INGENIERIA": "Ingeniería y Tecnología", "ARQUITECTURA": "Ingeniería y Tecnología",
    "SISTEMAS": "Ingeniería y Tecnología", "SOFTWARE": "Ingeniería y Tecnología",
    "COMPUTACION": "Ingeniería y Tecnología", "TELECOMUNICACIONES": "Ingeniería y Tecnología",
    "MECATRONICA": "Ingeniería y Tecnología", "ELECTRONICA": "Ingeniería y Tecnología",
    "MECANICA": "Ingeniería y Tecnología", "CIVIL": "Ingeniería y Tecnología",
    "TELEMATICA": "Ingeniería y Tecnología", "MULTIMEDIA": "Ingeniería y Tecnología",
    "URBANISMO": "Ingeniería y Tecnología", "CONSTRUCCION": "Ingeniería y Tecnología",
    "AERONAUTICA": "Ingeniería y Tecnología", "NANOTECNOLOGIA": "Ingeniería y Tecnología",
    "AUTOMATIZACION": "Ingeniería y Tecnología",
    # --- 4. Ciencias Económicas y Administrativas ---
    "ADMINISTRACION": "Ciencias Económicas y Administrativas", "ECONOMIA": "Ciencias Económicas y Administrativas",
    "CONTADURIA": "Ciencias Económicas y Administrativas", "FINANZAS": "Ciencias Económicas y Administrativas",
    "FINANCIERA": "Ciencias Económicas y Administrativas", "MERCADEO": "Ciencias Económicas y Administrativas",
    "PUBLICIDAD": "Ciencias Económicas y Administrativas", "NEGOCIOS": "Ciencias Económicas y Administrativas",
    "COMERCIO": "Ciencias Económicas y Administrativas", "HOTELERIA": "Ciencias Económicas y Administrativas",
    "TURISMO": "Ciencias Económicas y Administrativas", "LOGISTICA": "Ciencias Económicas y Administrativas",
    "BANCARIA": "Ciencias Económicas y Administrativas", "INDUSTRIAL": "Ciencias Económicas y Administrativas",
    # --- 5. Ciencias Sociales y Humanas ---
    "DERECHO": "Ciencias Sociales y Humanas", "JURISPRUDENCIA": "Ciencias Sociales y Humanas",
    "CIENCIA POLITICA": "Ciencias Sociales y Humanas", "TRABAJO SOCIAL": "Ciencias Sociales y Humanas",
    "SOCIOLOGIA": "Ciencias Sociales y Humanas", "ANTROPOLOGIA": "Ciencias Sociales y Humanas",
    "HISTORIA": "Ciencias Sociales y Humanas", "FILOSOFIA": "Ciencias Sociales y Humanas",
    "RELACIONES INTERNACIONALES": "Ciencias Sociales y Humanas", "GEOGRAFIA": "Ciencias Sociales y Humanas",
    "HUMANIDADES": "Ciencias Sociales y Humanas", "SOCIALES": "Ciencias Sociales y Humanas",
    "POLITICA": "Ciencias Sociales y Humanas", "GOBIERNO": "Ciencias Sociales y Humanas",
    "CRIMINALISTICA": "Ciencias Sociales y Humanas", "ARCHIVISTICA": "Ciencias Sociales y Humanas",
    "BIBLIOTECOLOGIA": "Ciencias Sociales y Humanas", "LENGUAS": "Ciencias Sociales y Humanas",
    "FILOLOGIA": "Ciencias Sociales y Humanas", "LETRAS": "Ciencias Sociales y Humanas",
    "LITERATURA": "Ciencias Sociales y Humanas", "LINGUISTICA": "Ciencias Sociales y Humanas",
    # --- 6. Artes, Diseño y Comunicación ---
    "DISEÑO": "Artes, Diseño y Comunicación", "COMUNICACION": "Artes, Diseño y Comunicación",
    "PERIODISMO": "Artes, Diseño y Comunicación", "ARTE": "Artes, Diseño y Comunicación",
    "ARTES": "Artes, Diseño y Comunicación", "MUSICA": "Artes, Diseño y Comunicación",
    "CINE": "Artes, Diseño y Comunicación", "AUDIOVISUAL": "Artes, Diseño y Comunicación",
    "FOTOGRAFIA": "Artes, Diseño y Comunicación", "DANZA": "Artes, Diseño y Comunicación",
    "TEATRO": "Artes, Diseño y Comunicación", "GASTRONOMIA": "Artes, Diseño y Comunicación",
    "CULINARIA": "Artes, Diseño y Comunicación", "MODAS": "Artes, Diseño y Comunicación",
    # --- 7. Ciencias Básicas y Naturales ---
    "BIOLOGIA": "Ciencias Básicas y Naturales", "QUIMICA": "Ciencias Básicas y Naturales",
    "FISICA": "Ciencias Básicas y Naturales", "MATEMATICAS": "Ciencias Básicas y Naturales",
    "ESTADISTICA": "Ciencias Básicas y Naturales", "GEOLOGIA": "Ciencias Básicas y Naturales",
    "ECOLOGIA": "Ciencias Básicas y Naturales", "ASTRONOMIA": "Ciencias Básicas y Naturales",
    "MICROBIOLOGIA": "Ciencias Básicas y Naturales", "GEOCIENCIAS": "Ciencias Básicas y Naturales",
    "BIOTECNOLOGIA": "Ciencias Básicas y Naturales", "BIOQUIMICA": "Ciencias Básicas y Naturales",
    # --- 8. Ciencias Agropecuarias y Ambientales ---
    "AGRONOMIA": "Ciencias Agropecuarias y Ambientales", "AGROPECUARIA": "Ciencias Agropecuarias y Ambientales",
    "AGROINDUSTRIAL": "Ciencias Agropecuarias y Ambientales", "AGRONEGOCIOS": "Ciencias Agropecuarias y Ambientales",
    "PECUARIA": "Ciencias Agropecuarias y Ambientales", "ACUICULTURA": "Ciencias Agropecuarias y Ambientales",
    "FORESTAL": "Ciencias Agropecuarias y Ambientales", "AGRICOLA": "Ciencias Agropecuarias y Ambientales",
    "AMBIENTAL": "Ciencias Agropecuarias y Ambientales",
    # --- 9. Otras Áreas ---
    "MILITARES": "Otras Áreas", "TEOLOGIA": "Otras Áreas", "NAVALES": "Otras Áreas",
    "NAUTICAS": "Otras Áreas", "DEPORTE": "Otras Áreas", "RECREACION": "Otras Áreas",
}

#Elimina tildes, espacios extra y convierte a mayúsculas para estandarizar.
def normalizar_texto(texto):
  # Si el valor no es texto, retorna un valor por defecto para evitar errores
    if not isinstance(texto, str): return "DESCONOCIDO"
    #Eliminación de tildes y caracteres especiales
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    #Pasa todo a mayúsculas y elimina espacios en blanco al inicio y al final
    return texto.upper().strip()

def asignar_area(programa):
  # Limpia el nombre del programa antes de buscar coincidencias
    prog_norm = normalizar_texto(programa)
    # Recorre cada palabra clave y su área en el diccionario
    for keyword, area in MAPEO_AREAS.items():
      # Si la palabra clave está dentro del nombre del programa...
        if keyword in prog_norm:
          # retorna el área correspondiente inmediatamente
            return area
    # Si termina el bucle sin encontrar nada, asigna la categoría "Otras áreas"
    return "Otras Áreas"

# Se aplica la lógica al DataFrame

# Rellena valores vacíos y normaliza la columna de programas académicos
df_final['E_PRGM_ACADEMICO'] = df_final['E_PRGM_ACADEMICO'].fillna("DESCONOCIDO").apply(normalizar_texto)

# Crea una nueva columna clasificando cada programa en su Área de conocimiento
df_final['F_AREA_CON_PRGM'] = df_final['E_PRGM_ACADEMICO'].apply(asignar_area)

# Creación de nuevas columnas y limpieza de valores NaN de las columnas

In [163]:
# Relleno de nulos con 'no info'
cols_fillna = ['E_PRGM_DEPARTAMENTO', 'F_TIENEINTERNET', 'F_TIENECOMPUTADOR',
               'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO']
for col in cols_fillna:
    df_final[col] = df_final[col].fillna('no info')

# Conversión de columnas de si tiene internet y computador a valores numéricos
map_binario = {'Si': 1, 'No': 0, 'no info': 0}
df_final['TIENE_INTERNET_NUM'] = df_final['F_TIENEINTERNET'].map(map_binario)
df_final['TIENE_PC_NUM'] = df_final['F_TIENECOMPUTADOR'].map(map_binario)

# Creación de nueva columna sumando los valores entre si tiene internet y computador
df_final['SCORE_TECNOLOGICO'] = df_final['TIENE_INTERNET_NUM'] + df_final['TIENE_PC_NUM']

# Conversión de columnas de educación del nivel de educación del padre y de la madre a valores numéricos
map_educacion_score = {
    'Ninguno': 0, 'No sabe': 0, 'No Aplica': 0, 'no info': 0,
    'Primaria incompleta': 1, 'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7, 'Educación profesional completa': 8, 'Postgrado': 9
}
df_final['F_EDUCACIONPADRE_NUM'] = df_final['F_EDUCACIONPADRE'].map(map_educacion_score).fillna(0)
df_final['F_EDUCACIONMADRE_NUM'] = df_final['F_EDUCACIONMADRE'].map(map_educacion_score).fillna(0)

# Creación de nueva columna sumando los valores entre el nivel educativo de la madre y el padre
df_final['CAPITAL_EDUCATIVO'] = df_final['F_EDUCACIONPADRE_NUM'] + df_final['F_EDUCACIONMADRE_NUM']

# Mapeo Ordinal de Educación (Para usar como variable directa)
df_final['F_EDUCACIONPADRE'] = df_final['F_EDUCACIONPADRE'].map(map_educacion_score).fillna(-1)
df_final['F_EDUCACIONMADRE'] = df_final['F_EDUCACIONMADRE'].map(map_educacion_score).fillna(-1)

# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango del
# valor de la matrícula universitaria
map_matricula = {
 'No pagó matrícula': 0,
 'Menos de 500 mil': .25,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'Más de 7 millones': 7.75,
 'no info': -1}
df_final['E_VALORMATRICULAUNIVERSIDAD'] = df_final['E_VALORMATRICULAUNIVERSIDAD'].map(map_matricula).fillna(-1)

# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango
# del número de horas trabajadas por los estudiantes
map_horasTrabajo = {
 '0': 0,
 'Menos de 10 horas': 5,
 'Entre 11 y 20 horas': 15.5,
 'Entre 21 y 30 horas': 25.5,
 'Más de 30 horas': 35.5,
 'no info': -1}
df_final['E_HORASSEMANATRABAJA'] = df_final['E_HORASSEMANATRABAJA'].map(map_horasTrabajo).fillna(-1)

# Definición de variables y separación de las mismas en numéricas y categóricas

In [164]:
# Definición de variables númericas
numeric_features = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4',
                    'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA','F_EDUCACIONPADRE',
                    'F_EDUCACIONMADRE', 'VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3', 'PERIODO_ACADEMICO',
                    'CAPITAL_EDUCATIVO', 'SCORE_TECNOLOGICO', 'TIENE_PC_NUM', 'TIENE_INTERNET_NUM'
                    ]

# Definición de variables categóricas
categorical_features = ['E_PRGM_DEPARTAMENTO', 'F_AREA_CON_PRGM','F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
                        'E_PAGOMATRICULAPROPIO']

# Definición de variable con alta cardinalidad
high_card_features = ['E_PRGM_ACADEMICO']

# Definición de variable objetivo y variables predictoras

In [165]:
target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
y = df_final['RENDIMIENTO_GLOBAL'].map(target_map)
X = df_final.drop(columns=['RENDIMIENTO_GLOBAL'])

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

 # Pipeline y entrenamiento del modelo

In [166]:
# Procesamiento de datos escalando números y codificando categorías
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('high_card', TargetEncoder(target_type='continuous', smooth=10.0), high_card_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)
    ],
    verbose_feature_names_out=False
)

# Pipeline con hiperparámetros definidos con RandomizedSearchCV
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_iter=1000,
        max_leaf_nodes=63,
        max_depth=None,
        l2_regularization=0,
        random_state=42,
        class_weight='balanced'
    ))
])

print("Entrenando modelo...")
model_pipeline.fit(X_train, y_train)

Entrenando modelo...


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['INDICADOR_1', 'INDICADOR_2',
                                                   'INDICADOR_3', 'INDICADOR_4',
                                                   'E_VALORMATRICULAUNIVERSIDAD',
                                                   'E_HORASSEMANATRABAJA',
                                                   'F_EDUCACIONPADRE',
                                                   'F_EDUCACIONMADRE',
                                                   'VAR_EXTRA_1', 'VAR_EXTRA_2',
                                                   'VAR_EXTRA_3',
                                                   'PERIODO_ACADEMICO',
                                                   'CAPITAL_EDUCATIVO',
                                                   'SCORE_TECNOLOGICO',
                                                   'TIENE_PC_NU...
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['E_PRGM_DEPARTAMENTO',
                                                   'F_AREA_CON_PRGM',
                                                   'F_TIENELAVADORA',
                                                   'F_TIENEAUTOMOVIL',
                                                   'E_PAGOMATRICULAPROPIO'])],
                                   verbose_feature_names_out=False)),
                ('classifier',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                l2_regularization=0,
                                                learning_rate=0.05,
                                                max_iter=1000,
                                                max_leaf_nodes=63,
                                                random_state=42))])

# Evaluación de resultados

Se verifica qué tan bien aprendió el modelo, se imprime Accuracy y un reporte detallado por clase

In [167]:
print("\nResultados en Test:")
y_pred = model_pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nReporte:")
print(classification_report(y_test, y_pred))




Resultados en Test:
Accuracy: 0.4404

Reporte:
              precision    recall  f1-score   support

           0       0.47      0.58      0.52     34597
           1       0.34      0.28      0.30     34455
           2       0.34      0.29      0.31     34324
           3       0.57      0.62      0.59     35124

    accuracy                           0.44    138500
   macro avg       0.43      0.44      0.43    138500
weighted avg       0.43      0.44      0.43    138500



#**SE MANTIENE EL PRIMERR MODELO; AHORA SE APLICA EL MISMO PROCEDIMIENTO A `test.csv`**

- Se debe conservar los ID para poder crear correctamente el archivo de envío

# Se agrega al dataframe train la desviación estándar, puntaje  y la cantidad de estudiantes por programa académico y por departamento que presentaron las pruebas saber pro entre 2018 y 2021

In [168]:
import pandas as pd
import unicodedata

"""
 id del archivo que contiene la desviación estándar, el puntaje promedio y la cantidad de estudiantes por programa académico y por departamento
 que presentaron las pruebas saber pro entre 2018 y 2021, los datos extraidos de los enlaces de abajo fueron filtrados y consolidados en un solo
 archivo csv que se integró como tres nuevas columnas al dataset.

 Enlaces a los archivos con los que se construyó el consolidado
 https://www.icfes.gov.co/wp-content/uploads/2024/12/Resultados-agregados-saber-pro-2018-1.xlsx
 https://www.icfes.gov.co/wp-content/uploads/2024/12/Base-de-datos-de-resultados-agregados-de-saber-pro-2019-1.xlsx
 https://www.icfes.gov.co/wp-content/uploads/2025/06/Base-de-datos-de-resultados-agregados-de-saber-pro-2020.xlsx
 https://www.icfes.gov.co/wp-content/uploads/2025/06/Base-de-datos-de-resultados-agregados-de-saber-pro-2021.xlsx
"""
file_id = '1ZAkIYpc0w0pAiER0OotuW0uyljl-1a_I'

# Url del archivo con los datos adicionales
url_datos_agregados = f'https://drive.google.com/uc?export=download&id={file_id}'

#Elimina tildes, espacios extra y convierte a mayúsculas para estandarizar.
def normalizar_texto(texto):
  # Si el valor no es texto, retorna un valor por defecto para evitar errores
    if not isinstance(texto, str):
      #Devuelve el valor por defecto si no es texto
        return "DESCONOCIDO"
    #Eliminación de tildes y caracteres especiales
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    #Pasa todo a mayúsculas y elimina espacios en blanco al inicio y al final
    return texto.upper().strip()

# Carga el archivo 'train.csv' en un DataFrame separando los datos por comas
df_train = pd.read_csv('test.csv', sep=',')

# Este archivo no tiene cabeceras, así que se asignan nombres manuales
col_names_consolidado = ['ANIO_CONSOL', 'DEPTO_CONSOL', 'PROG_CONSOL', 'VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3']

# Carga datos desde la URL y asigna nombres manuales, ya que el archivo no tiene cabeceras
df_consol = pd.read_csv(url_datos_agregados, header=None, names=col_names_consolidado)

# Extrae los primeros 4 dígitos (el año) del periodo y los convierte a entero
df_train['ANIO_MATCH'] = df_train['PERIODO_ACADEMICO'].astype(str).str[:4].astype(int)

# Aplica la función de limpieza para estandarizar los nombres del departamento en el dataframe train
df_train['DEPTO_MATCH'] = df_train['E_PRGM_DEPARTAMENTO'].apply(normalizar_texto)

# Aplica la función de limpieza para estandarizar los nombres del programa académico en el dataframe train
df_train['PROG_MATCH'] = df_train['E_PRGM_ACADEMICO'].apply(normalizar_texto)

# Aplica la función de limpieza para estandarizar los nombres del departamento en el dataframe consolidado que contiene los datos adicionales
df_consol['DEPTO_MATCH'] = df_consol['DEPTO_CONSOL'].apply(normalizar_texto)

# Aplica la función de limpieza para estandarizar los nombres del programa academico en el dataframe consolidado que contiene los datos adicionales
df_consol['PROG_MATCH'] = df_consol['PROG_CONSOL'].apply(normalizar_texto)

# Convierte el año a entero para asegurar la coincidencia en el cruce
df_consol['ANIO_MATCH'] = df_consol['ANIO_CONSOL'].astype(int)

# Cruza las tablas (Left Join) usando las claves generadas para agregar las variables del archivo consolidado
df_merged = pd.merge(
    df_train,
    df_consol[['ANIO_MATCH', 'DEPTO_MATCH', 'PROG_MATCH', 'VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3']],
    on=['ANIO_MATCH', 'DEPTO_MATCH', 'PROG_MATCH'],
    how='left'
)

# Elimina las columnas auxiliares que se crearon solo para realizar el cruce
df_final = df_merged.drop(columns=['ANIO_MATCH', 'DEPTO_MATCH', 'PROG_MATCH'])

print(f"Dimensiones de df_final antes de eliminar duplicados de ID: {df_final.shape}")
# Eliminar filas duplicadas basadas en 'ID' para asegurar IDs únicos para la submission
df_final.drop_duplicates(subset='ID', keep='first', inplace=True)

# Mostrar resultados
print(f"Dimensiones de df_final después de eliminar duplicados de ID: {df_final.shape}")
print(f"Dimensiones originales: {df_train.shape}")
print(f"Dimensiones finales (tras limpieza y deduplicación): {df_final.shape}")
print("Columnas nuevas agregadas:", ['VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3'])
print(df_final.head())

Dimensiones de df_final antes de eliminar duplicados de ID: (383881, 23)
Dimensiones de df_final después de eliminar duplicados de ID: (296786, 23)
Dimensiones originales: (296786, 23)
Dimensiones finales (tras limpieza y deduplicación): (296786, 23)
Columnas nuevas agregadas: ['VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3']
       ID  PERIODO_ACADEMICO                        E_PRGM_ACADEMICO  \
0  550236              20183                          TRABAJO SOCIAL   
1   98545              20203  ADMINISTRACION COMERCIAL Y DE MERCADEO   
2  499179              20212                  INGENIERIA MECATRONICA   
3  782980              20195                      CONTADURIA PUBLICA   
4  785185              20212              ADMINISTRACION DE EMPRESAS   

  E_PRGM_DEPARTAMENTO               E_VALORMATRICULAUNIVERSIDAD  \
0             BOLIVAR                          Menos de 500 mil   
1           ANTIOQUIA  Entre 2.5 millones y menos de 4 millones   
2              BOGOTÁ    Entre 1 millón y 

# Agrupación por áreas de conocimiento de la columna PROGRAMA_ACADEMICO
Para reducir la complejidad de la columna PROGRAMA_ACADEMICO (que tiene demasiados valores únicos), se crea un diccionario que agrupa las carreras en 9 áreas (educación, Ciencias de la Salud, Ingeniería y Tecnología, Ciencias Sociales y Humanas, Artes, Diseño y Comunicación, Ciencias Básicas y Naturales, Ciencias Agropecuarias y Ambientales y Otras Áreas)

In [169]:
# --- DICCIONARIO DE ÁREAS ---
MAPEO_AREAS = {
    # --- 1. Educación ---
    "LICENCIATURA": "Educación (Licenciaturas)", "PEDAGOGIA": "Educación (Licenciaturas)",
    "ETNOEDUCACION": "Educación (Licenciaturas)",
    # --- 2. Ciencias de la Salud ---
    "ENFERMERIA": "Ciencias de la Salud", "MEDICINA": "Ciencias de la Salud",
    "ODONTOLOGIA": "Ciencias de la Salud", "PSICOLOGIA": "Ciencias de la Salud",
    "SALUD": "Ciencias de la Salud", "BACTERIOLOGIA": "Ciencias de la Salud",
    "FISIOTERAPIA": "Ciencias de la Salud", "NUTRICION": "Ciencias de la Salud",
    "INSTRUMENTACION QUIRURGICA": "Ciencias de la Salud", "OPTOMETRIA": "Ciencias de la Salud",
    "FONOAUDIOLOGIA": "Ciencias de la Salud", "TERAPIA": "Ciencias de la Salud",
    "VETERINARIA": "Ciencias de la Salud", "ZOOTECNIA": "Ciencias de la Salud",
    "BIOINGENIERIA": "Ciencias de la Salud", "BIOMEDICA": "Ciencias de la Salud",
    "GERONTOLOGIA": "Ciencias de la Salud",
    # --- 3. Ingeniería y Tecnología ---
    "INGENIERIA": "Ingeniería y Tecnología", "ARQUITECTURA": "Ingeniería y Tecnología",
    "SISTEMAS": "Ingeniería y Tecnología", "SOFTWARE": "Ingeniería y Tecnología",
    "COMPUTACION": "Ingeniería y Tecnología", "TELECOMUNICACIONES": "Ingeniería y Tecnología",
    "MECATRONICA": "Ingeniería y Tecnología", "ELECTRONICA": "Ingeniería y Tecnología",
    "MECANICA": "Ingeniería y Tecnología", "CIVIL": "Ingeniería y Tecnología",
    "TELEMATICA": "Ingeniería y Tecnología", "MULTIMEDIA": "Ingeniería y Tecnología",
    "URBANISMO": "Ingeniería y Tecnología", "CONSTRUCCION": "Ingeniería y Tecnología",
    "AERONAUTICA": "Ingeniería y Tecnología", "NANOTECNOLOGIA": "Ingeniería y Tecnología",
    "AUTOMATIZACION": "Ingeniería y Tecnología",
    # --- 4. Ciencias Económicas y Administrativas ---
    "ADMINISTRACION": "Ciencias Económicas y Administrativas", "ECONOMIA": "Ciencias Económicas y Administrativas",
    "CONTADURIA": "Ciencias Económicas y Administrativas", "FINANZAS": "Ciencias Económicas y Administrativas",
    "FINANCIERA": "Ciencias Económicas y Administrativas", "MERCADEO": "Ciencias Económicas y Administrativas",
    "PUBLICIDAD": "Ciencias Económicas y Administrativas", "NEGOCIOS": "Ciencias Económicas y Administrativas",
    "COMERCIO": "Ciencias Económicas y Administrativas", "HOTELERIA": "Ciencias Económicas y Administrativas",
    "TURISMO": "Ciencias Económicas y Administrativas", "LOGISTICA": "Ciencias Económicas y Administrativas",
    "BANCARIA": "Ciencias Económicas y Administrativas", "INDUSTRIAL": "Ciencias Económicas y Administrativas",
    # --- 5. Ciencias Sociales y Humanas ---
    "DERECHO": "Ciencias Sociales y Humanas", "JURISPRUDENCIA": "Ciencias Sociales y Humanas",
    "CIENCIA POLITICA": "Ciencias Sociales y Humanas", "TRABAJO SOCIAL": "Ciencias Sociales y Humanas",
    "SOCIOLOGIA": "Ciencias Sociales y Humanas", "ANTROPOLOGIA": "Ciencias Sociales y Humanas",
    "HISTORIA": "Ciencias Sociales y Humanas", "FILOSOFIA": "Ciencias Sociales y Humanas",
    "RELACIONES INTERNACIONALES": "Ciencias Sociales y Humanas", "GEOGRAFIA": "Ciencias Sociales y Humanas",
    "HUMANIDADES": "Ciencias Sociales y Humanas", "SOCIALES": "Ciencias Sociales y Humanas",
    "POLITICA": "Ciencias Sociales y Humanas", "GOBIERNO": "Ciencias Sociales y Humanas",
    "CRIMINALISTICA": "Ciencias Sociales y Humanas", "ARCHIVISTICA": "Ciencias Sociales y Humanas",
    "BIBLIOTECOLOGIA": "Ciencias Sociales y Humanas", "LENGUAS": "Ciencias Sociales y Humanas",
    "FILOLOGIA": "Ciencias Sociales y Humanas", "LETRAS": "Ciencias Sociales y Humanas",
    "LITERATURA": "Ciencias Sociales y Humanas", "LINGUISTICA": "Ciencias Sociales y Humanas",
    # --- 6. Artes, Diseño y Comunicación ---
    "DISEÑO": "Artes, Diseño y Comunicación", "COMUNICACION": "Artes, Diseño y Comunicación",
    "PERIODISMO": "Artes, Diseño y Comunicación", "ARTE": "Artes, Diseño y Comunicación",
    "ARTES": "Artes, Diseño y Comunicación", "MUSICA": "Artes, Diseño y Comunicación",
    "CINE": "Artes, Diseño y Comunicación", "AUDIOVISUAL": "Artes, Diseño y Comunicación",
    "FOTOGRAFIA": "Artes, Diseño y Comunicación", "DANZA": "Artes, Diseño y Comunicación",
    "TEATRO": "Artes, Diseño y Comunicación", "GASTRONOMIA": "Artes, Diseño y Comunicación",
    "CULINARIA": "Artes, Diseño y Comunicación", "MODAS": "Artes, Diseño y Comunicación",
    # --- 7. Ciencias Básicas y Naturales ---
    "BIOLOGIA": "Ciencias Básicas y Naturales", "QUIMICA": "Ciencias Básicas y Naturales",
    "FISICA": "Ciencias Básicas y Naturales", "MATEMATICAS": "Ciencias Básicas y Naturales",
    "ESTADISTICA": "Ciencias Básicas y Naturales", "GEOLOGIA": "Ciencias Básicas y Naturales",
    "ECOLOGIA": "Ciencias Básicas y Naturales", "ASTRONOMIA": "Ciencias Básicas y Naturales",
    "MICROBIOLOGIA": "Ciencias Básicas y Naturales", "GEOCIENCIAS": "Ciencias Básicas y Naturales",
    "BIOTECNOLOGIA": "Ciencias Básicas y Naturales", "BIOQUIMICA": "Ciencias Básicas y Naturales",
    # --- 8. Ciencias Agropecuarias y Ambientales ---
    "AGRONOMIA": "Ciencias Agropecuarias y Ambientales", "AGROPECUARIA": "Ciencias Agropecuarias y Ambientales",
    "AGROINDUSTRIAL": "Ciencias Agropecuarias y Ambientales", "AGRONEGOCIOS": "Ciencias Agropecuarias y Ambientales",
    "PECUARIA": "Ciencias Agropecuarias y Ambientales", "ACUICULTURA": "Ciencias Agropecuarias y Ambientales",
    "FORESTAL": "Ciencias Agropecuarias y Ambientales", "AGRICOLA": "Ciencias Agropecuarias y Ambientales",
    "AMBIENTAL": "Ciencias Agropecuarias y Ambientales",
    # --- 9. Otras Áreas ---
    "MILITARES": "Otras Áreas", "TEOLOGIA": "Otras Áreas", "NAVALES": "Otras Áreas",
    "NAUTICAS": "Otras Áreas", "DEPORTE": "Otras Áreas", "RECREACION": "Otras Áreas",
}

#Elimina tildes, espacios extra y convierte a mayúsculas para estandarizar.
def normalizar_texto(texto):
  # Si el valor no es texto, retorna un valor por defecto para evitar errores
    if not isinstance(texto, str): return "DESCONOCIDO"
    #Eliminación de tildes y caracteres especiales
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    #Pasa todo a mayúsculas y elimina espacios en blanco al inicio y al final
    return texto.upper().strip()

def asignar_area(programa):
  # Limpia el nombre del programa antes de buscar coincidencias
    prog_norm = normalizar_texto(programa)
    # Recorre cada palabra clave y su área en el diccionario
    for keyword, area in MAPEO_AREAS.items():
      # Si la palabra clave está dentro del nombre del programa...
        if keyword in prog_norm:
          # retorna el área correspondiente inmediatamente
            return area
    # Si termina el bucle sin encontrar nada, asigna la categoría "Otras áreas"
    return "Otras Áreas"

# Se aplica la lógica al DataFrame

# Rellena valores vacíos y normaliza la columna de programas académicos
df_final['E_PRGM_ACADEMICO'] = df_final['E_PRGM_ACADEMICO'].fillna("DESCONOCIDO").apply(normalizar_texto)

# Crea una nueva columna clasificando cada programa en su Área de conocimiento
df_final['F_AREA_CON_PRGM'] = df_final['E_PRGM_ACADEMICO'].apply(asignar_area)

# Creación de nuevas columnas y limpieza de valores NaN de las columnas

In [170]:
# Relleno de nulos con 'no info'
cols_fillna = ['E_PRGM_DEPARTAMENTO', 'F_TIENEINTERNET', 'F_TIENECOMPUTADOR',
               'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO']
for col in cols_fillna:
    df_final[col] = df_final[col].fillna('no info')

# Conversión de columnas de si tiene internet y computador a valores numéricos
map_binario = {'Si': 1, 'No': 0, 'no info': 0}
df_final['TIENE_INTERNET_NUM'] = df_final['F_TIENEINTERNET'].map(map_binario)
df_final['TIENE_PC_NUM'] = df_final['F_TIENECOMPUTADOR'].map(map_binario)

# Creación de nueva columna sumando los valores entre si tiene internet y computador
df_final['SCORE_TECNOLOGICO'] = df_final['TIENE_INTERNET_NUM'] + df_final['TIENE_PC_NUM']

# Conversión de columnas de educación del nivel de educación del padre y de la madre a valores numéricos
map_educacion_score = {
    'Ninguno': 0, 'No sabe': 0, 'No Aplica': 0, 'no info': 0,
    'Primaria incompleta': 1, 'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7, 'Educación profesional completa': 8, 'Postgrado': 9
}
df_final['F_EDUCACIONPADRE_NUM'] = df_final['F_EDUCACIONPADRE'].map(map_educacion_score).fillna(0)
df_final['F_EDUCACIONMADRE_NUM'] = df_final['F_EDUCACIONMADRE'].map(map_educacion_score).fillna(0)

# Creación de nueva columna sumando los valores entre el nivel educativo de la madre y el padre
df_final['CAPITAL_EDUCATIVO'] = df_final['F_EDUCACIONPADRE_NUM'] + df_final['F_EDUCACIONMADRE_NUM']

# Mapeo Ordinal de Educación (Para usar como variable directa)
df_final['F_EDUCACIONPADRE'] = df_final['F_EDUCACIONPADRE'].map(map_educacion_score).fillna(-1)
df_final['F_EDUCACIONMADRE'] = df_final['F_EDUCACIONMADRE'].map(map_educacion_score).fillna(-1)

# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango del
# valor de la matrícula universitaria
map_matricula = {
 'No pagó matrícula': 0,
 'Menos de 500 mil': .25,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'Más de 7 millones': 7.75,
 'no info': -1}
df_final['E_VALORMATRICULAUNIVERSIDAD'] = df_final['E_VALORMATRICULAUNIVERSIDAD'].map(map_matricula).fillna(-1)

# Diccionario de phyton que mapea las categorías de texto a un número flotante que represente el punto medio del rango
# del número de horas trabajadas por los estudiantes
map_horasTrabajo = {
 '0': 0,
 'Menos de 10 horas': 5,
 'Entre 11 y 20 horas': 15.5,
 'Entre 21 y 30 horas': 25.5,
 'Más de 30 horas': 35.5,
 'no info': -1}
df_final['E_HORASSEMANATRABAJA'] = df_final['E_HORASSEMANATRABAJA'].map(map_horasTrabajo).fillna(-1)

# Definición de variables y separación de las mismas en numéricas y categóricas

In [171]:
# Definición de variables númericas
numeric_features = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4',
                    'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA','F_EDUCACIONPADRE',
                    'F_EDUCACIONMADRE', 'VAR_EXTRA_1', 'VAR_EXTRA_2', 'VAR_EXTRA_3', 'PERIODO_ACADEMICO',
                    'CAPITAL_EDUCATIVO', 'SCORE_TECNOLOGICO', 'TIENE_PC_NUM', 'TIENE_INTERNET_NUM'
                    ]

# Definición de variables categóricas
categorical_features = ['E_PRGM_DEPARTAMENTO', 'F_AREA_CON_PRGM','F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
                        'E_PAGOMATRICULAPROPIO']

# Definición de variable con alta cardinalidad
high_card_features = ['E_PRGM_ACADEMICO']

# Aplicación del pipeline al conjunto de datos de prueba `test.csv`

In [172]:
# Reordena las columnas alfabéticamente y mantiene el formato DataFrame para que el Pipeline reconozca los nombres de las variables
X_test_data = df_final[sorted(df_final.columns)]

# Genera las predicciones finales (las etiquetas de clase) sobre el nuevo conjunto de datos usando el modelo ya entrenado
preds_test_data = model_pipeline.predict(X_test_data)


Resultados en Test:


# Preparación de la entrega a Kaggle

In [173]:
# Define el nombre de la columna objetivo
y_col = 'RENDIMIENTO_GLOBAL'

# Diccionario de la variable objetivo
rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}

# Extraer los IDs de los estudiantes del archivo de test para asegurar el orden correcto
ids = df_final['ID'].values

# Invierte el diccionario: Ahora las claves son números y los valores texto
rmapi = {v:k for k,v in rmap.items()}

# Traduce las predicciones numéricas del modelo de vuelta a etiquetas de texto
text_preds_test_data = [rmapi[i] for i in preds_test_data]

# Crea el DataFrame uniendo IDs y Predicciones, y lo transpone (.T) para tener el formato correcto (filas x columnas)
submission = pd.DataFrame([ids, text_preds_test_data], index=['ID', 'RENDIMIENTO_GLOBAL']).T

# Verificación
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


# Preparar archivo CSV

In [174]:
# Guarda el resultado en un archivo CSV, omitiendo el índice (números de fila) para cumplir con el formato de entrega
submission.to_csv("my_submission.csv", index=False)

# Verificación
submission.shape

(296786, 2)

# Envío de archivo de predicciones a la competencia de Kaggle

In [175]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f my_submission.csv -m "Entrega actualizada Pablo Uribe"

100% 4.07M/4.07M [00:00<00:00, 9.78MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia